In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
class ConvertToObject:
    '''
    Класс объектов, представляющий собой объединённую базу данных из считанных файлов
    Доступны методы:
    to_xlsx - считывание файла с расширением .xls

    Атрибуты класса:
    self.path_files - пути к считываемым файлам
    self.name_files - имена считываемых файлов
    self.dict_obj - массив, хранящий считанные данные
    '''

    def to_xlsx(self, path_file, index = -1):
        # get path of file and open file in xls. After put in dict
        data = pd.read_excel(path_file)

        #data = data.iloc[2:, :]
        #data.drop(data.columns[[1,2,3]], axis=1 ,inplace=True)
        
        #data.index = data.iloc[:, 0].tolist()
        #data.drop(data.columns[[0]], axis=1 ,inplace=True)
        
        # ставим год наверх
        #data.columns = data.iloc[0, :].astype(int).tolist()
        #data = data.iloc[1:, :]

        if index == -1:
            self.dict_obj[path_file] = data
        else:
            self.dict_obj[self.name_files[index]] = data
    
    def __init__(self):
        self.path_files = []
        self.name_files = []
        self.dict_obj = dict()


        to_directory = os.getcwd() + "/Data/data_xls/"
        self.name_files = os.listdir(to_directory)

        for i in range(len(self.name_files)):
            self.path_files.append(os.path.join(to_directory, self.name_files[i]))
            
            end_of_namefiles = self.name_files[i][-4:]

            if end_of_namefiles == "xlsx":
                self.to_xlsx(self.path_files[i], i)

In [3]:
# Создаём объект

obj = ConvertToObject()

In [4]:
# Задаём копию считанных данных

db = obj.dict_obj.copy()

In [5]:
# Задаём массив имён файлов в директории

path_dir = os.getcwd() + "\\Data\\data_xls\\"

file_list = os.listdir(path_dir)

In [8]:
for i in range(0, 18):
    sample = db[file_list[i]]
    sample.index = sample.iloc[:, 0].tolist()
    sample.drop(sample.columns[[0]], axis=1 ,inplace=True)
    sample.columns = sample.columns.map(int)

In [9]:
GLOBAL_DATASET_YEAR = dict()

YEAR = [2015, 2016, 2017, 2018, 2019]

for year in YEAR:
    GLOBAL_DATASET_YEAR[year] = pd.DataFrame()
    for name in obj.name_files:
        end_of_name = name[-4:]
        if end_of_name == "xlsx":
            temp = obj.dict_obj[name][year].rename(name[:-5])
            GLOBAL_DATASET_YEAR[year] = pd.concat([GLOBAL_DATASET_YEAR[year], temp], axis=1)
    #GLOBAL_DATASET_YEAR[year] = pd.concat([GLOBAL_DATASET_YEAR[year], sample_3[year].rename(file_list[0])], axis=1)

In [10]:
temp = list(set(GLOBAL_DATASET_YEAR[year].index.tolist()))

for year in YEAR:
    GLOBAL_DATASET_YEAR[year] = GLOBAL_DATASET_YEAR[year].loc[temp, :]
    GLOBAL_DATASET_YEAR[year].sort_index(inplace=True)

In [11]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(GLOBAL_DATASET_YEAR[2015])

                          agricultural_land  agriculture_value_added  \
Albania                              42.858                   19.780   
Algeria                              17.406                   11.579   
Angola                               36.728                    9.123   
Armenia                              58.894                   17.223   
Australia                            45.314                    2.375   
Austria                              32.943                    1.130   
Bahamas, The                          1.299                    0.827   
Bangladesh                           72.476                   14.783   
Belarus                              42.242                    6.283   
Belgium                              43.857                    0.686   
Belize                                7.453                    9.607   
Benin                                35.030                   26.392   
Bhutan                               13.789                   14

In [12]:
# Задаём заголовок для стран и удаляем "выбросы"

for year in YEAR:
    GLOBAL_DATASET_YEAR[year].index.name = "COUNTRY"

In [13]:
# Приписываем год элементам

for year in YEAR:
    GLOBAL_DATASET_YEAR[year]["YEAR"] = year*np.ones(len(GLOBAL_DATASET_YEAR[year]), dtype=int)

In [14]:
# Меняем пропущенные значения на среднее по столбцу

for year in YEAR:
    GLOBAL_DATASET_YEAR[year].fillna(GLOBAL_DATASET_YEAR[year].mean(), inplace = True)
    GLOBAL_DATASET_YEAR[year].drop(['West Bank and Gaza','Togo','Timor-Leste','Sierra Leone','Rwanda',
                                    'Namibia','Nepal','Mauritius','Lesotho','Lebanon','Gabon','Gambia, The',
                                    'Eswatini','El Salvador','Djibouti',"Cote d'Ivoire",'Cabo Verde','Burundi',
                                    'Brunei Darussalam','Bhutan','Benin','Belize','Bahamas, The','Vanuatu','Bolivia', 'Kuwait', 'Hong Kong SAR, China'], axis=0, inplace=True)

C:\Users\verei\AppData\Local\Temp\ipykernel_12280\2468951179.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  GLOBAL_DATASET_YEAR[year].fillna(GLOBAL_DATASET_YEAR[year].mean(), inplace = True)


In [15]:
# "Склеиваем" датасет и сортируем по странам и годам

GLOBAL_DATASET = pd.concat(GLOBAL_DATASET_YEAR.values())
GLOBAL_DATASET = GLOBAL_DATASET.sort_values(["COUNTRY", "YEAR"])

In [16]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
    print(GLOBAL_DATASET)

                          agricultural_land agriculture_value_added  \
COUNTRY                                                               
Albania                           42.857664               19.780214   
Albania                           43.127737               19.849994   
Albania                           42.856971               19.022127   
Albania                           42.849672               18.429495   
Albania                           42.846715               18.389332   
Algeria                           17.405923               11.578707   
Algeria                           17.365532               12.220095   
Algeria                           17.355011               11.756089   
Algeria                           17.356568               11.874372   
Algeria                           17.347004               12.336953   
Angola                            36.728162                9.122534   
Angola                            36.756236                9.831169   
Angola

In [17]:
GLOBAL_DATASET

,agricultural_land,agriculture_value_added,cereal_yield,consumption,crop_index,employment_agriculture,export,FDI,GDP,import,inflation_cost_prices,inflation_deflator,livestock_index,natural_resources_rent,rural_population,total_tax,trade,urban_population,YEAR
COUNTRY,,,,,,,,,,,,,,,,,,,
Albania,42.857664,19.780214,4873.8,1.182727,99.91,41.283525,27.267391,9.895783e+08,1.138685e+10,44.533615,1.896174,0.563991,100.24,1.734862,42.566,36.9,71.801006,57.434,2015
Albania,43.127737,19.849994,4716.4,2.126820,104.15,40.040852,28.977899,1.044390e+09,1.176433e+10,45.831964,1.275432,-0.632653,101.53,1.058481,41.579,36.9,74.809863,58.421,2016
Albania,42.856971,19.022127,4813.0,2.443446,105.84,38.078346,31.569798,1.022758e+09,1.221168e+10,46.624451,1.986661,1.450732,100.97,1.346042,40.617,37.3,78.194250,59.383,2017
Albania,42.849672,18.429495,4840.5,3.215538,106.17,37.285732,31.571954,1.204383e+09,1.270250e+10,45.236230,2.028060,1.472953,105.50,1.711116,39.681,37.3,76.808184,60.319,2018
Albania,42.846715,18.389332,5038.2,3.303659,107.67,36.416856,31.304270,1.201022e+09,1.296770e+10,44.974925,1.411091,1.257025,98.98,1.634974,38.771,36.6,76.279195,61.229,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zimbabwe,41.876696,8.284247,557.5,19.080902,96.35,66.267404,19.160176,3.992000e+08,1.996312e+10,37.588635,-2.430968,0.367420,102.30,4.606185,67.615,31.6,56.748811,32.385,2015
Zimbabwe,41.876696,7.873986,435.1,-10.501114,94.14,65.366319,19.943532,3.430138e+08,2.014298e+10,31.275493,-1.543670,2.014095,103.85,4.495414,67.704,31.6,51.219025,32.296,2016
Zimbabwe,41.876696,8.340969,1203.3,1.396143,107.07,64.573919,19.658905,3.071877e+08,2.096487e+10,30.370807,0.893962,3.056905,101.98,6.095448,67.763,31.6,50.029712,32.237,2017


In [18]:
# Сохраняем в файл

GLOBAL_DATASET.to_csv(os.getcwd()+"/global_data.csv", index=True, sep=';')